# Завантажити файл з yourtube та надіслати завдання на обробку завантаженого відео обробнику video-Worker прямо шляхом публікації повідомленя в Redis

---
В даномій задачі Jupyter Notebooks  використовуються як інструменти тестування, відлагодження та моніторингу.

**На приклад:**
В уілях відладки ми можемо покласти якийсь сторонній файл у каталог **video** вказати його найменуваня в повідомленні для redis і подивитися результат роботи worker в логах:

```bash
docker compose logs video-worker -f

```



In [17]:
# Імпортуємо пакети
import os
import json
import time
import yt_dlp

import redis
from rq import Queue, Worker

# Читаю параметри підключення до Redis з env змінних
irds_host = os.getenv('RDS_HOST')
irds_port = os.getenv('RDS_PORT')
irds_psw = os.getenv('RDS_PSW')
irdsq_outmsg = os.getenv('RDSQ_OUTMSG')

# для debug
#print(f"Прочита значення змінних підключеня до Redis {irds_host} {irds_port} {irds_psw} {irdsq_outmsg}")

# Функція завантаження відео
def download_youtube_video(url, output_path='.', obj_name='video'):
    ydl_opts = {
        # Змінюємо пріоритет: шукаємо саме h264 (avc1), який є стандартом для OpenCV
        'format': 'bestvideo[vcodec^=avc1][ext=mp4]+bestaudio[ext=m4a]/best[vcodec^=avc1][ext=mp4]/best',
        'outtmpl': f'{output_path}/{obj_name}.%(ext)s',
        'noplaylist': True,
        # Додаємо merge_output_format, щоб на виході точно був mp4
        'merge_output_format': 'mp4',
    }
    
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        print(f"Завантаження відео з URL: {url}...")
        ydl.download([url])
        print("Завантаження завершено!")
    
    return f'{obj_name}.mp4'



## Підключаюся до  Redis

In [18]:
# Саме підключення
red = redis.StrictRedis(irds_host, irds_port, password=irds_psw, decode_responses=False)

# Пробую  Ping
rping=red.ping()
print( f"Статус підключення: {rping} ")
q = None
q_msg = None

# Аналізуємо значення ping
if rping:
    print("До Redis підключидлися успішно")
    q = Queue(connection=red)
    q_msg = Queue( name=irdsq_outmsg, connection=red)
    print(f"Задач у черзі {irdsq_outmsg}: {len(q_msg)}")
else:
    print(" НE ПІДКЛЮЧИЛИСЯ ДО REDIS!!!")    

Статус підключення: True 
До Redis підключидлися успішно
Задач у черзі video_queue: 0


## Завантаження відео

In [19]:
# готую список URL для завантаження

# Список завантажених файлів
file_names=[]

# Список підготованих URL для завантаження

"""
test_videos = {
    "cats": "https://www.youtube.com/watch?v=hmw-w4PFms8",
    "cats_2": "https://www.youtube.com/shorts/jyzmRnSpOJc" ,
    "cats_3":"https://www.youtube.com/shorts/EM41yq0OUQ4",
    "man_1": "https://www.youtube.com/shorts/W1CWGIe-5ts"
}
"""



test_videos = {
    "cats": "https://www.youtube.com/watch?v=hmw-w4PFms8",
    "cats_2": "https://www.youtube.com/shorts/jyzmRnSpOJc" ,
    "cats_3":"https://www.youtube.com/shorts/EM41yq0OUQ4",
    "man_1": "https://www.youtube.com/shorts/W1CWGIe-5ts",
    "man_2": "https://www.youtube.com/watch?v=YFyTwSoAjyI&list=RDYFyTwSoAjyI&start_radio=1",
    "tango": "https://www.youtube.com/watch?v=YFyTwSoAjyI"


}




# Створюємо папку для тестів, якщо її немає
output_directory = "video"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

print(f"--- Початок завантаження {len(test_videos)} відео для тестів ---")

for obj_name, url in test_videos.items():
    print(f"\nОбробка об'єкта: {obj_name}")
    try:
        filename=download_youtube_video(url, output_path=output_directory, obj_name=obj_name )
        #file_names.add(obj_name + '.mp4' )
        file_names.append(filename)
    except Exception as e:
        print(f"Помилка при завантаженні {obj_name}: {e}")

print("\n--- Всі завдання виконано! ---", file_names)


--- Початок завантаження 6 відео для тестів ---

Обробка об'єкта: cats
Завантаження відео з URL: https://www.youtube.com/watch?v=hmw-w4PFms8...
[youtube] Extracting URL: https://www.youtube.com/watch?v=hmw-w4PFms8
[youtube] hmw-w4PFms8: Downloading webpage


[youtube] hmw-w4PFms8: Downloading android vr player API JSON
[info] hmw-w4PFms8: Downloading 1 format(s): 134+140
[download] Destination: video/cats.f134.mp4
[download] 100% of  647.30KiB in 00:00:00 at 2.08MiB/s   
[download] Destination: video/cats.f140.m4a
[download] 100% of  476.30KiB in 00:00:00 at 4.55MiB/s   
[Merger] Merging formats into "video/cats.mp4"
Deleting original file video/cats.f134.mp4 (pass -k to keep)
Deleting original file video/cats.f140.m4a (pass -k to keep)
Завантаження завершено!

Обробка об'єкта: cats_2
Завантаження відео з URL: https://www.youtube.com/shorts/jyzmRnSpOJc...
[youtube] Extracting URL: https://www.youtube.com/shorts/jyzmRnSpOJc
[youtube] jyzmRnSpOJc: Downloading webpage


[youtube] jyzmRnSpOJc: Downloading android vr player API JSON
[info] jyzmRnSpOJc: Downloading 1 format(s): 136+140
[download] Destination: video/cats_2.f136.mp4
[download] 100% of    1.62MiB in 00:00:00 at 6.34MiB/s   
[download] Destination: video/cats_2.f140.m4a
[download] 100% of  237.50KiB in 00:00:00 at 3.31MiB/s     
[Merger] Merging formats into "video/cats_2.mp4"
Deleting original file video/cats_2.f140.m4a (pass -k to keep)
Deleting original file video/cats_2.f136.mp4 (pass -k to keep)
Завантаження завершено!

Обробка об'єкта: cats_3
Завантаження відео з URL: https://www.youtube.com/shorts/EM41yq0OUQ4...
[youtube] Extracting URL: https://www.youtube.com/shorts/EM41yq0OUQ4
[youtube] EM41yq0OUQ4: Downloading webpage


[youtube] EM41yq0OUQ4: Downloading android vr player API JSON
[info] EM41yq0OUQ4: Downloading 1 format(s): 137+140
[download] Destination: video/cats_3.f137.mp4
[download] 100% of    2.66MiB in 00:00:00 at 7.06MiB/s   
[download] Destination: video/cats_3.f140.m4a
[download] 100% of  170.44KiB in 00:00:00 at 1.00MiB/s   
[Merger] Merging formats into "video/cats_3.mp4"
Deleting original file video/cats_3.f137.mp4 (pass -k to keep)
Deleting original file video/cats_3.f140.m4a (pass -k to keep)
Завантаження завершено!

Обробка об'єкта: man_1
Завантаження відео з URL: https://www.youtube.com/shorts/W1CWGIe-5ts...
[youtube] Extracting URL: https://www.youtube.com/shorts/W1CWGIe-5ts
[youtube] W1CWGIe-5ts: Downloading webpage


[youtube] W1CWGIe-5ts: Downloading android vr player API JSON
[info] W1CWGIe-5ts: Downloading 1 format(s): 299+140
[download] Destination: video/man_1.f299.mp4
[download] 100% of   31.69MiB in 00:00:03 at 8.84MiB/s     
[download] Destination: video/man_1.f140.m4a
[download] 100% of  752.81KiB in 00:00:00 at 7.82MiB/s     
[Merger] Merging formats into "video/man_1.mp4"
Deleting original file video/man_1.f140.m4a (pass -k to keep)
Deleting original file video/man_1.f299.mp4 (pass -k to keep)
Завантаження завершено!

Обробка об'єкта: man_2
Завантаження відео з URL: https://www.youtube.com/watch?v=YFyTwSoAjyI&list=RDYFyTwSoAjyI&start_radio=1...
[youtube:tab] Extracting URL: https://www.youtube.com/watch?v=YFyTwSoAjyI&list=RDYFyTwSoAjyI&start_radio=1
[youtube:tab] Downloading just the video YFyTwSoAjyI because of --no-playlist
[youtube] Extracting URL: https://www.youtube.com/watch?v=YFyTwSoAjyI
[youtube] YFyTwSoAjyI: Downloading webpage


[youtube] YFyTwSoAjyI: Downloading android vr player API JSON
[info] YFyTwSoAjyI: Downloading 1 format(s): 137+140
[download] Destination: video/man_2.f137.mp4
[download] 100% of   56.30MiB in 00:00:06 at 8.91MiB/s     
[download] Destination: video/man_2.f140.m4a
[download] 100% of    2.92MiB in 00:00:00 at 7.65MiB/s   
[Merger] Merging formats into "video/man_2.mp4"
Deleting original file video/man_2.f137.mp4 (pass -k to keep)
Deleting original file video/man_2.f140.m4a (pass -k to keep)
Завантаження завершено!

Обробка об'єкта: tango
Завантаження відео з URL: https://www.youtube.com/watch?v=YFyTwSoAjyI...
[youtube] Extracting URL: https://www.youtube.com/watch?v=YFyTwSoAjyI
[youtube] YFyTwSoAjyI: Downloading webpage


[youtube] YFyTwSoAjyI: Downloading android vr player API JSON
[info] YFyTwSoAjyI: Downloading 1 format(s): 137+140
[download] Destination: video/tango.f137.mp4
[download] 100% of   56.30MiB in 00:00:06 at 8.30MiB/s     
[download] Destination: video/tango.f140.m4a
[download] 100% of    2.92MiB in 00:00:00 at 5.71MiB/s   
[Merger] Merging formats into "video/tango.mp4"
Deleting original file video/tango.f137.mp4 (pass -k to keep)
Deleting original file video/tango.f140.m4a (pass -k to keep)
Завантаження завершено!

--- Всі завдання виконано! --- ['cats.mp4', 'cats_2.mp4', 'cats_3.mp4', 'man_1.mp4', 'man_2.mp4', 'tango.mp4']


## Відправляємо повідмолення в Redis

In [20]:
#import json
print("Завантажую обробник черги....")
from utils import tasks
print("Завантаження виконано")
#file_name="20260225-171507.avi"
try:
    for file_name in file_names:
        # Публікуємо повідмолення в чергу для оборобки відое file_name
        message_o={"filename": file_name}
        message_s=json.dumps(message_o)
        print(f"Повідомлення для черги: {message_s}")
        job = q_msg.enqueue('utils.tasks.crttask_sendmsg',  message_s)
        print(f"Задачу додано в чергу: {job.id} для файлу {file_name}")
        # Список усіх ID завдань у черзі
        queued_job_ids = q_msg.job_ids
        print(f"ID завдань у черзі: {queued_job_ids}")

except Exception as e:
    print(f"Помилка черги Redis: {e}")

Завантажую обробник черги....
Завантаження виконано
Повідомлення для черги: {"filename": "cats.mp4"}
Задачу додано в чергу: aa84f07d-981d-48c1-af6e-0f7406f169df для файлу cats.mp4
ID завдань у черзі: []
Повідомлення для черги: {"filename": "cats_2.mp4"}
Задачу додано в чергу: 571fd1db-37d7-44be-a428-dcf491f413de для файлу cats_2.mp4
ID завдань у черзі: ['571fd1db-37d7-44be-a428-dcf491f413de']
Повідомлення для черги: {"filename": "cats_3.mp4"}
Задачу додано в чергу: 3fc91b50-6838-4073-bec0-58f72586e632 для файлу cats_3.mp4
ID завдань у черзі: ['571fd1db-37d7-44be-a428-dcf491f413de', '3fc91b50-6838-4073-bec0-58f72586e632']
Повідомлення для черги: {"filename": "man_1.mp4"}
Задачу додано в чергу: 98e90d4d-c8e4-45db-a61f-b9bea0386fac для файлу man_1.mp4
ID завдань у черзі: ['571fd1db-37d7-44be-a428-dcf491f413de', '3fc91b50-6838-4073-bec0-58f72586e632', '98e90d4d-c8e4-45db-a61f-b9bea0386fac']
Повідомлення для черги: {"filename": "man_2.mp4"}
Задачу додано в чергу: 39caa632-390f-407b-bfd8-5a3

## Відобразити лог обробки повідомлень

In [28]:
import pandas as pd
import matplotlib.pyplot as plt

# Читаємо файл
df = pd.read_json('/app/video/detections_log.jsonl', lines=True)

# Перетворюємо час у нормальний формат
df['timestamp'] = pd.to_datetime(df['timestamp'])

display(df.head(50))

,ok,filename,detected,count,timestamp
0,True,20260223-144657.avi,[],0,2026-02-23 14:47:56.697779
1,True,20260223-144819.avi,"[person, tv, microwave, oven, keyboard]",5,2026-02-23 14:49:15.581433
2,True,man_2.mp4,[],0,2026-02-26 10:40:37.041319
3,True,20260226-104219.avi,"[bed, vase, clock, person]",4,2026-02-26 10:43:17.913595
4,True,20260226-104249.avi,"[person, potted plant]",2,2026-02-26 10:43:44.238045
5,True,20260226-104811.avi,[potted plant],1,2026-02-26 10:48:59.488946
6,True,20260226-105615.avi,"[potted plant, person, tv]",3,2026-02-26 10:57:08.755341
7,True,20260226-110150.avi,[potted plant],1,2026-02-26 11:02:43.470789
8,True,20260226-110337.avi,[potted plant],1,2026-02-26 11:04:22.559744
9,True,20260226-110444.avi,[potted plant],1,2026-02-26 11:05:30.350700
